# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [15]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [16]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'deepseek/deepseek-chat'
openai = OpenAI()

API key looks good so far


In [17]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """
    
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
    
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [18]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-soft

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [19]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [20]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [21]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [22]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https:/

In [23]:
def get_links(url):
    website = Website(url)
    
    client = OpenAI(
        api_key=api_key,
        base_url="https://router.requesty.ai/v1"
        )
    
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [24]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/deepseek-ai/DeepSeek-R1',
 '/hexgrad/Kokoro-82M',
 '/deepseek-ai/DeepSeek-R1-Distill-Qwen-32B',
 '/openbmb/MiniCPM-o-2_6',
 '/deepseek-ai/DeepSeek-R1-Zero',
 '/models',
 '/spaces/hexgrad/Kokoro-TTS',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/FaceOnLive/Face-Search-Online',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/HumanLLMs/Human-Like-DPO-Dataset',
 '/datasets/yale-nlp/MMVU',
 '/datasets/bespokelabs/Bespoke-Stratos-17k',
 '/datasets/NovaSky-AI/Sky-T1_data_17k',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/doc

In [25]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'join page', 'url': 'https://huggingface.co/join'},
  {'type': 'docs page', 'url': 'https://huggingface.co/docs'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'learn page', 'url': 'https://huggingface.co/learn'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'github page', 'url': 'https://github.com/huggingface'},
  {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'linkedin page',
   'url': 'https://

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [26]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [27]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'chat page', 'url': 'https://huggingface.co/chat'}, {'type': 'tasks page', 'url': 'https://huggingface.co/tasks'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datas

In [28]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [29]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [30]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'models', 'url': 'https://huggingface.co/models'}, {'type': 'datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://huggingface.co/pricing'}, {'type': 'join', 'url': 'https://huggingface.co/join'}, {'type': 'docs', 'url': 'https://huggingface.co/docs'}, {'type': 'learn', 'url': 'https://huggingface.co/learn'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'brand', 'url': 'https://huggingface.co/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-R1\nUpdated\nabout 8 hours ago\n•\n44.6k\n•\n1.97k\nhexgrad/Kokoro-82M\nUpdated\n1 day ago\n•\n32.6k\n•\n2.28k\ndeepseek-ai/DeepSeek-R1-Distill-Qwen-32B\nUpdated\nabout 7 hours ago\n•\n63.7k\n•\n386\nopenbmb/MiniCPM-o-2_6\nUpdated\nabout 9 hours ago\n•\n50.8k\n•\n786\ndeepseek-ai/DeepSeek-R1-Zero\nUpdated\nabout 7 hours ago\n•\n3.04k\n•\n368\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n1.43k\n❤️\nKokoro TTS\

In [33]:
def create_brochure(company_name, url):
    
    client = OpenAI(
        api_key=api_key,
        base_url="https://router.requesty.ai/v1"
    )
    
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [34]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/'}, {'type': 'models page', 'url': 'https://huggingface.com/models'}, {'type': 'datasets page', 'url': 'https://huggingface.com/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.com/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'join page', 'url': 'https://huggingface.com/join'}, {'type': 'docs page', 'url': 'https://huggingface.com/docs'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'learn page', 'url': 'https://huggingface.com/learn'}, {'type': 'brand page', 'url': 'https://huggingface.com/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.lin

# Hugging Face: The AI Community Building the Future

## About Hugging Face
**Hugging Face** is a leading AI community and platform where machine learning enthusiasts, researchers, and developers collaborate to build the future of AI. The platform is designed to foster innovation by enabling users to create, discover, and collaborate on models, datasets, and applications across various modalities like text, image, video, audio, and even 3D.

### Mission
Hugging Face aims to democratize AI by providing tools and resources that make machine learning accessible to everyone. The platform is built on the principles of open collaboration, transparency, and innovation.

---

## Key Features
### **Models**
- Access over **400,000+ models** for various tasks, including state-of-the-art models for text, image, audio, and more.
- Collaborate on trending models like **DeepSeek-R1**, **Kokoro-82M**, and **MiniCPM-o-2_6**.

### **Datasets**
- Explore and share **100,000+ datasets** for computer vision, NLP, and audio tasks.
- Popular datasets include **awesome-chatgpt-prompts**, **Human-Like-DPO-Dataset**, and **MMVU**.

### **Spaces**
- Build and deploy **150,000+ applications** with ease.
- Examples include **Kokoro TTS**, **Hunyuan3D-2.0**, and **IC Light V2**.

### **Enterprise Solutions**
- Advanced AI platform with enterprise-grade security, access controls, and dedicated support.
- Starting at **$20/user/month** with features like Single Sign-On, Audit Logs, and Private Datasets Viewer.

### **Open Source Tools**
- Hugging Face is the home of popular open-source libraries like:
  - **Transformers**: State-of-the-art ML for PyTorch, TensorFlow, and JAX.
  - **Diffusers**: Diffusion models for image and audio generation.
  - **Tokenizers**: Fast tokenizers for research and production.
  - **Datasets**: Access and share datasets for various AI tasks.

---

## Who Uses Hugging Face?
Hugging Face is trusted by **50,000+ organizations**, including:
- **AI2** (non-profit)
- **AI at Meta**
- **Amazon Web Services**
- **Google**
- **Intel**
- **Microsoft**
- **Grammarly**
- **Writer**

---

## Careers at Hugging Face
Hugging Face is more than a platform; it’s a community of passionate individuals driving the future of AI. The company values innovation, collaboration, and inclusivity. If you’re looking to make an impact in the AI space, explore career opportunities at Hugging Face.

### Why Join?
- Work on cutting-edge AI technologies.
- Collaborate with a global community of AI enthusiasts.
- Contribute to open-source projects that shape the future of machine learning.

---

## Get Started
Whether you’re a researcher, developer, or enterprise, Hugging Face has something for you:
- **Sign Up** for free to start collaborating.
- Explore **Compute Solutions** starting at **$0.60/hour** for GPU.
- Join the community and build your ML portfolio.

---

**Hugging Face** is where the machine learning community comes together to innovate, collaborate, and build the future of AI. Join us today! 🚀

[Visit Hugging Face](https://huggingface.co) | [Explore Careers](https://huggingface.co/jobs) | [Sign Up](https://huggingface.co/join)

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [39]:
def stream_brochure(company_name, url):
    
    client = OpenAI(
        api_key=api_key,
        base_url="https://router.requesty.ai/v1"
    )
    
    
    stream = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [40]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}]}


# Hugging Face Brochure

## **The AI Community Building the Future**
Hugging Face is a leading platform where the machine learning (ML) community collaborates to create, discover, and share models, datasets, and applications. With a mission to democratize AI, Hugging Face empowers individuals and organizations to innovate and accelerate their ML projects.

---

### **Key Offerings**
- **Models**: Access over **400,000+ models** for text, image, video, audio, and 3D applications.
- **Datasets**: Explore **100,000+ datasets** for computer vision, NLP, and more.
- **Spaces**: Build and deploy **150,000+ applications** with ease.
- **Open Source Tools**: Leverage state-of-the-art libraries like **Transformers**, **Diffusers**, and **Tokenizers**.
- **Enterprise Solutions**: Advanced AI platforms with enterprise-grade security, dedicated support, and scalability.

---

### **Why Hugging Face?**
- **Collaborative Community**: Join a vibrant ecosystem of ML practitioners, researchers, and organizations.
- **Cutting-Edge Technology**: Stay ahead with the latest advancements in AI and ML.
- **Open Source Commitment**: Contribute to and benefit from open-source tools that power the future of AI.
- **Scalable Solutions**: From startups to enterprises, Hugging Face offers tailored solutions to meet your needs.

---

### **Customers**
Hugging Face is trusted by **50,000+ organizations**, including industry leaders like:
- **AI at Meta**
- **Google**
- **Microsoft**
- **Intel**
- **Amazon Web Services**
- **Grammarly**
- **Writer**

---

### **Careers**
Hugging Face is more than a platform—it’s a community of passionate individuals shaping the future of AI. Join us to:
- Work on **state-of-the-art ML technologies**.
- Collaborate with a **global, diverse team**.
- Contribute to **open-source projects** that impact millions.

**Current Openings**: Visit our [Careers Page](https://huggingface.co/careers) to explore opportunities.

---

### **Enterprise Solutions**
Hugging Face offers enterprise-grade AI solutions to help your team:
- **Accelerate Development**: Deploy models and applications faster with optimized tools.
- **Ensure Security**: Benefit from enterprise-grade security, access controls, and audit logs.
- **Scale with Confidence**: Dedicated support and scalable infrastructure for growing teams.

**Pricing**: Starting at **$20/user/month** for enterprise plans.

---

### **Join the Community**
- **Sign Up**: Start collaborating today at [Hugging Face](https://huggingface.co).
- **Explore**: Dive into models, datasets, and spaces.
- **Contribute**: Share your work and build your ML portfolio.

---

**Hugging Face** – Where the future of AI is built, together. 🌟

In [42]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}]}


# Hugging Face Brochure

## **The AI Community Building the Future**
Hugging Face is a leading platform where the machine learning (ML) community collaborates to build the future of AI. With a mission to democratize AI, Hugging Face provides tools, resources, and a collaborative environment for creating, discovering, and sharing ML models, datasets, and applications.

---

## **What We Offer**
### **For the ML Community**
- **Models**: Access over **400,000+ models** for text, image, video, audio, and 3D applications.
- **Datasets**: Explore **100,000+ datasets** to power your AI projects.
- **Spaces**: Build and share **150,000+ applications** with the community.
- **Open Source Tools**: Leverage state-of-the-art libraries like **Transformers**, **Diffusers**, **Tokenizers**, and more.

### **For Enterprises**
- **Enterprise Solutions**: Advanced AI platform with enterprise-grade security, access controls, and dedicated support.
  - **Starting at $20/user/month**
  - Features include Single Sign-On, Audit Logs, Resource Groups, and Private Datasets Viewer.
- **Compute Solutions**: Optimized Inference Endpoints and GPU-powered Spaces applications.
  - **Starting at $0.60/hour for GPU**

---

## **Our Community**
Hugging Face is trusted by **50,000+ organizations**, including:
- **AI2** (non-profit)
- **AI at Meta**
- **Amazon Web Services**
- **Google**
- **Intel**
- **Microsoft**
- **Grammarly**
- **Writer**

---

## **Company Culture**
Hugging Face is built on collaboration, innovation, and openness. We believe in empowering the ML community to create and share cutting-edge AI solutions. Our culture is rooted in:
- **Open Source**: We actively contribute to and maintain foundational ML tools.
- **Inclusivity**: We welcome diverse perspectives and foster a global community.
- **Learning**: We provide resources like documentation, forums, and blogs to help users grow.

---

## **Careers at Hugging Face**
Join us in shaping the future of AI! Hugging Face offers exciting opportunities for:
- **AI Researchers**
- **Software Engineers**
- **Data Scientists**
- **Product Managers**
- **Community Builders**

Visit our [Careers Page](#) to explore current openings and become part of a team that’s revolutionizing AI.

---

## **Why Choose Hugging Face?**
- **Collaboration**: Host and collaborate on unlimited public models, datasets, and applications.
- **Speed**: Accelerate your ML projects with our open-source stack and optimized compute solutions.
- **Diversity**: Explore all modalities—text, image, video, audio, and 3D.
- **Portfolio Building**: Share your work with the world and build your ML profile.

---

## **Get Started**
- **Sign Up**: Join the Hugging Face community today.
- **Enterprise Hub**: Discover how Hugging Face can empower your organization.
- **Compute Solutions**: Deploy your models with ease.

---

**Hugging Face** – Where the ML community collaborates to build the future of AI. 🌟

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>